In [1]:
# https://gseapy.readthedocs.io/en/latest/gseapy_example.html

In [2]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import gseapy as gp

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
dbs = gp.get_library_name()


## Read in naming file

In [5]:
inputFile = 'data/omics_LLM_DF.tsv'

In [6]:
genesets_LLM = pd.read_csv(inputFile, sep = "\t") 

In [7]:
# Remove "HALLMARK_" prefix for MSigDB gene sets
genesets_LLM['GeneSetName'] = genesets_LLM['GeneSetName'].str.removeprefix("HALLMARK_").str.replace('_', ' ')


In [8]:
genesets_LLM.columns

Index(['Source', 'GeneSetName', 'GeneList', 'n_Genes', 'LLM Name',
       'LLM Analysis', 'Score'],
      dtype='object')

In [9]:
columnsToKeep = ['Source', 'GeneSetID', 'GeneSetName', 'GeneList', 'n_Genes', 'LLM Name',
       'LLM Analysis', 'Score']
geneSeparator = " "

In [10]:
LLM_DF = genesets_LLM

In [11]:
nGeneSets = LLM_DF.shape[0]

In [12]:
nGeneSets

225

## Go through each system and run Enrichr 

In [13]:
# Each system will have multiple rows

In [14]:
additionalCols = ['Rank', 'Overlap', 'P-value', 'Adjusted P-value', 'Genes', 'GO term', 'GO ID' ]

In [15]:
expandedColumnsToKeep =columnsToKeep;
expandedColumnsToKeep.extend(additionalCols)

In [16]:
expanded_LLM_genes_DF = pd.DataFrame(columns=expandedColumnsToKeep)

In [17]:
genesCol = 'GeneList'

In [18]:
for geneSetInd in range(nGeneSets):
    print(geneSetInd)
    geneSetRow = LLM_DF.iloc[geneSetInd]
    geneSetGenes = geneSetRow[genesCol].split(geneSeparator)

    # Perform enrichment analysis
    enr = gp.enrichr(gene_list=geneSetGenes, # or "./tests/data/gene_list.txt",
                 gene_sets=['GO_Biological_Process_2023'],
                 organism='human', # don't forget to set organism to the one you desired! e.g. Yeast
                 outdir=None, # don't write to disk
                )
    
    # threshold
    # sigRes_DF =  enr.results.loc[enr.results['Adjusted P-value'] <=0.1]
    sigRes_DF = enr.results # remove thresholding, because want a name for each one
    sigRes_DF = sigRes_DF.sort_values('Adjusted P-value')
    if sigRes_DF.shape[0] == 0:
        
        print([geneSetInd, ' has not enirchment'])
        #continue # skip to next gene set
        sigRes_DF.loc[0, ['GO term', 'GO ID', 'nothing']] = 'None'
        sigRes_DF.loc[0, ['Rank']]  = 1
    else:
        expandedCols = sigRes_DF["Term"].str.split('(', expand = True)
        
        if expandedCols.shape[1] == 3:
            sigRes_DF[['GO term', 'GO ID', 'nothing']] = expandedCols
        else: # ==2
            sigRes_DF[['GO term', 'GO ID']] = expandedCols

        sigRes_DF[['GO ID', 'nothing']] = sigRes_DF["GO ID"].str.split(')', expand = True)

        # Get ranking of enriched GO terms
        sigRes_DF['Rank'] = sigRes_DF.reset_index().index

    
    # Doing what should be a simple colbind 
    geneSetRow_DF = geneSetRow.to_frame().T
    geneSetRow_repeated_DF = geneSetRow_DF.loc[geneSetRow_DF.index.repeat(sigRes_DF.shape[0])]
    geneSetRow_repeated_DF = geneSetRow_repeated_DF.reset_index()
    
    # system_Row_sigRes = systemRow_repeated_DF.join(sigRes_DF)
    geneSetRowRow_sigRes = pd.concat([geneSetRow_repeated_DF, sigRes_DF],axis=1)
    
    # Doing what should be a simple  rowbind
    if expanded_LLM_genes_DF.shape[0] == 0:
        expanded_LLM_genes_DF = geneSetRowRow_sigRes[expandedColumnsToKeep]
    else:
        expanded_LLM_genes_DF =  pd.concat([expanded_LLM_genes_DF, geneSetRowRow_sigRes[expandedColumnsToKeep]],axis=0)
    

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
[103, ' has not enirchment']
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224


In [21]:
expanded_LLM_genes_DF.shape

(107286, 14)

In [22]:
outputFile = 'data/omics_LLM_Enrichr_DF.tsv' 

In [23]:
expanded_LLM_genes_DF.to_csv(outputFile, sep = "\t")